In [1]:
#rectified flow from https://github.com/grahamclyne/RectifiedFlow/tree/main

#load score based model AND datasets

#generate 

from ipsl_dcpp.model.ipsl_dataset import IPSL_DCPP
import torch
import lightning as pl
from ipsl_dcpp.model.pangu import PanguWeather
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra
import numpy as np
import os
os.environ['SLURM_NTASKS_PER_NODE'] = '1'
with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
pl.seed_everything(cfg.experiment.seed)

train = hydra.utils.instantiate(
    cfg.experiment.train_dataset,
    generate_statistics=False,
    surface_variables=cfg.experiment.surface_variables,
    depth_variables=cfg.experiment.depth_variables,
    plev_variables=cfg.experiment.plev_variables,
    work_path=cfg.environment.work_path,
    scratch_path=cfg.environment.scratch_path,
)
train_dataloader = torch.utils.data.DataLoader(
    train,
    batch_size=1,
    shuffle=False,
    num_workers=1
)

checkpoint_path = torch.load(f'epoch=30.ckpt',map_location=torch.device('cpu'))
model = hydra.utils.instantiate(
    cfg.experiment.module,
    backbone=hydra.utils.instantiate(cfg.experiment.backbone),
    dataset=train
)

Seed set to 0
9it [00:01,  5.25it/s]
/Users/gclyne/miniforge3/envs/env_dcpp/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025535429/work/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
import ipsl_dcpp.sde_lib as sde_lib
sde = sde_lib.RectifiedFlow(init_type='gaussian', noise_scale=1, use_ode_sampler='rk45')
sampling_eps = 1e-3

Init. Distribution Variance: 1
SDE Sampler Variance: 0.0
ODE Tolerence: 1e-05


In [3]:
import ipsl_dcpp.losses as losses
import ml_collections
config = ml_collections.ConfigDict()

config.optim = optim = ml_collections.ConfigDict()
optim.weight_decay = 0.
optim.optimizer = 'Adam'
optim.lr = 2e-4
optim.beta1 = 0.9
optim.eps = 1e-8
optim.warmup = 5000
optim.grad_clip = 1.

optimize_fn = losses.optimization_manager(config)

train_step_fn = losses.get_step_fn(sde, train=True, optimize_fn=optimize_fn,
                                    reduce_mean=False, continuous=False,
                                    likelihood_weighting=False)

In [4]:
from ipsl_dcpp.ema import ExponentialMovingAverage
#for step in range(initial_step, num_train_steps + 1):
train_iter = iter(train_dataloader)
ema = ExponentialMovingAverage(model.parameters(), decay=0.999999)
optimizer = losses.get_optimizer(config, model.parameters())
state = dict(optimizer=optimizer, model=model, ema=ema, step=0)


In [5]:
for step in range(0, 10):

  # Convert data to JAX arrays and normalize them. Use ._numpy() to avoid copy.
  batch = next(train_iter)
  # batch = torch.from_numpy(next(train_iter)['image']._numpy()).to(config.device).float()
  # batch = batch.permute(0, 3, 1, 2)
  # batch = scaler(batch)
  # Execute one training step
  loss = train_step_fn(state, batch)
  print(loss)
    

torch.Size([1, 9, 143, 144])
torch.Size([1])
tensor([0.6372])


IndexError: too many indices for tensor of dimension 4